In [6]:
import pandas as pd
import numpy as np
import gc
import warnings
import time
from math import log
from datetime import timedelta
warnings.filterwarnings("ignore")

# kernel = 'kaggle'
kernel = 'local'
path = '../input/' if kernel == 'kaggle' else 'input\\'

catalog = {'train': 'sales_train.csv',
           'test': 'test.csv',
           'items': 'items.csv',
           'categories': 'item_categories.csv',
           'shops': 'shops.csv',
}
catalog = {k: path+v for k, v in catalog.items()}

class Logger:
    def __init__(self):
        self.start = time.time()
        self.last = self.start
        
    def lap(self, *sections):
        end = time.time()
        lap_time = timedelta(seconds=int(end-self.last)) 
        total_time = timedelta(seconds=int(end-self.start))
        self.last = end
        print('total: '+str(total_time)
              +' | section: '+str(lap_time)
              +' | '+ ' | '.join(sections))

l = Logger()

# Preprocessing

1) train & test

In [ ]:
# train & test
from datetime import datetime

train = pd.read_csv(catalog['train'])
train['date'] = pd.to_datetime(train['date'],  format='%d.%m.%Y')
train['quarter'] = train['date'].dt.quarter
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year

test = pd.read_csv(catalog['test'])
test['date'] = datetime(day=1,month=11,year=2015)
test['quarter'] = test['date'].dt.quarter
test['month'] = test['date'].dt.month
test['year'] = test['date'].dt.year


def _compare(*cols):
    cols, nm = list(cols), ",".join(cols)
    df1, df2 = train[cols].drop_duplicates(), test[cols].drop_duplicates()
    df3 = df1.merge(df2, on=cols, how='outer', indicator=True)
    v_count, v_count.name = df3['_merge'].value_counts(), nm
    v_count.index = v_count.index.map({'both': 'both', 
                                       'left_only': 'train_only', 
                                       'right_only': 'test_only'})
    print(v_count, '\n')
    del df1, df2, df3, v_count
    gc.collect()
    
_compare('shop_id')
_compare('item_id')
_compare('item_id', 'shop_id')

print('last block ', train['date_block_num'].max())
print('last date  ', train['date'].max())

l.lap('train & test')

2) Outliers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# boxplot to identify extreme values
ax1 = plt.subplot(211)
sns.boxplot(train.item_cnt_day, ax=ax1)
plt.title('item_cnt_day')
ax1 = plt.subplot(212)
sns.boxplot(train.item_price, ax=ax1)

# Clip prices and item_counts to more reasonable levels
train['item_cnt_day'] = train['item_cnt_day'].clip(0, 1001)
train['item_price'] = train['item_price'].clip(0, 50000)

3) combine train & test

In [ ]:
# Train
train['ID'] = -1

# Fix Test
test['date_block_num'] = 34
test['item_cnt_day'] = 0
test['item_price'] = 0
# Combine
combo = pd.concat([train, test])

del train, test
_ = gc.collect()

In [ ]:
combo.isnull().sum(axis=0)

4) grid - shop_id x item_id for each date_block_num

In [ ]:
def get_grid(df):
    from itertools import product
    cols = ['date_block_num', 'shop_id', 'item_id']
    grid = []
    for block in df['date_block_num'].unique():
        shops = df.loc[df['date_block_num']==block]['shop_id'].unique()
        items = df.loc[df['date_block_num']==block]['item_id'].unique()
        grid.append(np.array(list(product([block], shops, items))))
    grid = pd.DataFrame(np.vstack(grid), columns=cols)
    return grid

grid = get_grid(combo)

l.lap('preprocessing')

# Feature Engineering

1) month features

In [ ]:
def get_weekdays(month, year):
    start = datetime(day=1, month=month, year=year)
    
    year = year if month != 12 else year + 1
    month = month + 1 if month != 12 else 1
    end = datetime(day=1, month=month, year=year)
    return pd.date_range(start, end, closed='left' ).weekday.value_counts().sort_index().tolist()

month_feat = combo[['date_block_num', 'month', 'year', 'quarter']].drop_duplicates()
weekdays = month_feat.apply(lambda x: get_weekdays(x['month'], x['year']), axis=1)
weekdays = pd.DataFrame.from_items(zip(weekdays.index, weekdays.values)).T
weekdays.columns = ['wk_'+str(w) for w in weekdays if w != 'date_block_num']
month_feat = pd.concat([month_feat, weekdays], axis=1)
month_feat.drop('year', axis=1, inplace=True)

grid = grid.merge(month_feat, how='left')

month_feat = ['month', 'quarter']+['wk_'+str(i) for i in range(7)]

l.lap('month features')

2) item_category_id

In [ ]:
items = pd.read_csv(catalog['items'])
items.drop('item_name', axis=1, inplace=True)
# Merge
grid = grid.merge(items, on='item_id', how='left')
combo = combo.merge(items, on='item_id', how='left')

del items
gc.collect()
l.lap('item_category_id')

3) item_category_group_1, item_category_group_2

In [ ]:
# most category names are of the form 'group_1 - group_2'
cats = pd.read_csv(catalog['categories'])
foo = lambda x, idx: x.split(' - ')[idx] if len(x.split(' - ')) > idx else ''
cats['item_category_group_1'] = cats['item_category_name'].map(lambda x: foo(x, 0))
cats['item_category_group_2'] = cats['item_category_name'].map(lambda x: foo(x, 1))
cats['item_category_group_1'] = pd.factorize(cats['item_category_group_1'])[0]
cats['item_category_group_2'] = pd.factorize(cats['item_category_group_2'])[0]
cats.drop('item_category_name', axis=1, inplace=True)
# Merge
grid = grid.merge(cats, on='item_category_id', how='left')
combo = combo.merge(cats, on='item_category_id', how='left')

del cats
gc.collect()
l.lap('item_category_groups')

4) shop_group

In [ ]:
# First word of shop name is repeated
shops = pd.read_csv(catalog['shops'])
foo = lambda x, idx: x.split(' ')[idx] if len(x.split(' ')) > idx else ''
shops['shop_group'] = shops['shop_name'].map(lambda x: foo(x, 0))
shops['shop_group'] = pd.factorize(shops['shop_group'])[0]
shops.drop('shop_name', axis=1, inplace=True)
# Merge
grid = grid.merge(shops, on='shop_id', how='left')
combo = combo.merge(shops, on='shop_id', how='left')

del shops
gc.collect()
l.lap('shop_group')

In [ ]:
# Categorical columns
cat_feat = ['item_id', 'shop_id', 'item_category_id',
            'item_category_group_1', 'item_category_group_2', 'shop_group',]

5) monthly aggregates

In [ ]:
def get_item_month_cnt(df, *group_by):
    nm = "monthly_by_"+"_".join(group_by)
    cols = list(group_by) + ['date_block_num']
    cnt = combo.groupby(cols, as_index=False)['item_cnt_day'].agg({nm: sum})
    df = df.merge(cnt, how='left', on=cols+['date_block_num'])
    df[nm] = df[nm].fillna(0)
    del cnt
    gc.collect()
    l.lap('montly aggregates', ", ".join(group_by))
    return df
    
# target
grid = get_item_month_cnt(grid, 'shop_id', 'item_id')

# target column
target = 'monthly_by_shop_id_item_id'
grid[target] = grid[target].clip(0, 20)

# Monthly aggregate columns
monthly_feat = [c for c in grid if 'monthly_by_' in c]

6) mean encodings

In [ ]:
def get_mean_encoding(df, *group_by):
    nm = "average_by_"+"_".join(group_by)
    cols = list(group_by) + ['date_block_num']
    t_mean = df.groupby(cols, as_index=False)[target].agg({nm: 'mean'})
    df = df.merge(t_mean, on=cols, how='left')
    df[nm] = df[nm].fillna(0)
    del t_mean
    gc.collect()
    l.lap('mean encoding', ", ".join(group_by))
    return df

# Level 1
grid = get_mean_encoding(grid)
grid = get_mean_encoding(grid, 'shop_id')  
grid = get_mean_encoding(grid, 'item_id')  
grid = get_mean_encoding(grid, 'item_category_id') 
grid = get_mean_encoding(grid, 'item_category_group_1') 
grid = get_mean_encoding(grid, 'item_category_group_2')
grid = get_mean_encoding(grid, 'shop_group')
grid = get_mean_encoding(grid, 'shop_id', 'item_category_group_1')
grid = get_mean_encoding(grid, 'shop_id', 'item_category_group_2')
grid = get_mean_encoding(grid, 'shop_group', 'item_id')

# Monthly average columns
mean_enc_cols = [c for c in grid if 'average_by_' in c]

8) lag features function

In [ ]:
def get_lagged_features(grid, 
                        shift_range, 
                        lag_cols, 
                        index_cols=['shop_id', 'item_id', 'date_block_num']):
    for shift in shift_range:
        grid_shift = grid[index_cols+lag_cols].copy()
        grid_shift['date_block_num'] = grid_shift['date_block_num'] + shift
        foo = lambda x: '{}_lag_{}'.format(x, shift) if x in lag_cols else x
        grid_shift = grid_shift.rename(columns=foo)
        grid = pd.merge(grid, grid_shift, on=index_cols, how='left').fillna(0)
        l.lap('lagged features', str(shift))
    return grid

9) item_price_trend

In [ ]:
mn1 = combo.groupby('item_id', as_index=False)['item_price'].agg({'global': 'mean'})
mn2 = combo.groupby(['date_block_num', 'item_id'], 
                    as_index=False)['item_price'].agg({'month': 'mean'})
mn3 = mn2.merge(mn1, on='item_id', how='left')
mn3 = get_lagged_features(mn3, range(1,7), ['month'], ['item_id', 'date_block_num'])

lag_cols = [col for col in mn3.columns if '_lag_' in col]
for col in lag_cols:
    mn3[col] = (mn3[col]-mn3['month'])/mn3['global']
    
foo = lambda x: x[np.where(x!=0)[0][0]] if len(np.where(x!=0)[0]) != 0 else 0

mn3['item_price_trend'] = mn3[lag_cols].apply(foo, axis=1)
mn3['item_price_trend'] = mn3['item_price_trend'].fillna(0)
mn3 = mn3[['date_block_num', 'item_id', 'item_price_trend']]

grid = grid.merge(mn3, on=['date_block_num', 'item_id'], how='left')

del mn1, mn2, mn3
gc.collect()

l.lap('item_price_trend')

10) shop_revenue_trend

In [ ]:
combo['shop_revenue'] = combo['item_price'].astype('float64') * combo['item_cnt_day']
mn2 = (combo
       .loc[combo['date_block_num']<34]
       .groupby(['shop_id', 'date_block_num'], as_index=False)
       ['shop_revenue']
       .agg({'month': 'sum'}))
mn1 = mn2.groupby('shop_id', as_index=False)['month'].agg({'global': 'mean'})
mn3 = mn2.merge(mn1, on='shop_id', how='left')
mn3 = get_lagged_features(mn3, range(1,7), ['month'], ['shop_id', 'date_block_num'])

lag_cols = [col for col in mn3.columns if '_lag_' in col]
for col in lag_cols:
    mn3[col] = (mn3[col]-mn3['month'])/mn3['global']
    
foo = lambda x: x[np.where(x!=0)[0][0]] if len(np.where(x!=0)[0]) != 0 else 0

mn3['shop_revenue_trend'] = mn3[lag_cols].apply(foo, axis=1)
mn3['shop_revenue_trend'] = mn3['shop_revenue_trend'].fillna(0)
mn3 = mn3[['date_block_num', 'shop_id', 'shop_revenue_trend']]

grid = grid.merge(mn3, on=['date_block_num', 'shop_id'], how='left')

del mn1, mn2, mn3
gc.collect()

l.lap('shop_revenue_trend')

In [ ]:
grid.info()

11) downcasting dtypes

In [ ]:
# grid is 2gbs of memory, downcasting floats and ints to reduce to a quarter
def downcast_dtypes(df, cols=None):
    cols = cols if cols else df.columns.tolist()
    int_cols = df.select_dtypes(include=['int'+str(b) for b in [8,16,32,64]]).columns
    float_cols = df.select_dtypes(include=['float'+str(b) for b in [16,32,64]]).columns
    int_cols = [c for c in int_cols if c in cols]
    float_cols = [c for c in float_cols if c in cols]
    for col in int_cols:
        log_max = log(df[col].abs().max(), 2)
        base = min([v for v in [8,16,32,64] if v > log_max])
        dtype = 'int'+str(base)
        if df[col].dtype.name != dtype:
            df[col] = df[col].astype(dtype)
            
    for col in float_cols:
        log_max = log(df[col].abs().max(), 2)
        base = min([v for v in [16,32,64] if v > log_max])
        dtype = 'float'+str(base)
        if df[col].dtype.name != dtype:
            df[col] = df[col].astype(dtype)
    return df

grid = downcast_dtypes(grid)
l.lap('downcast')

In [ ]:
grid.info(verbose=False)

In [ ]:
print('Zeros %')
round((grid == 0).sum()/grid.shape[0]*100).sort_values(ascending=False)

11) lagged features

In [ ]:
idx =['shop_id', 'item_id', 'date_block_num']

grid = get_lagged_features(grid, [1,2,3,6,12], mean_enc_cols, idx)
grid = get_lagged_features(grid, [1], ['item_price_trend', 'shop_revenue_trend'], idx)
grid.drop(mean_enc_cols+['item_price_trend', 'shop_revenue_trend'], axis=1, inplace=True)
grid = grid.loc[grid['date_block_num'] >= 12]

In [ ]:
grid.info()

del combo
gc.collect()

# Model Evaluation

12) save/load grid

In [21]:
import pandas as pd
import gc

grid = pd.read_hdf('intermediate//grid', 'grid')
target = 'monthly_by_shop_id_item_id'
exclude = [
#     'month',
    'quarter',
    'wk_0', 
    'wk_1', 
    'wk_2', 
    'wk_3', 
    'wk_4', 
    'wk_5', 
    'wk_6'
]
features = grid.columns.difference([target, 'date_block_num']+exclude)
dates = grid['date_block_num']
X = grid[features]
y = grid[target]

del grid
gc.collect()
exclude

['quarter', 'wk_0', 'wk_1', 'wk_2', 'wk_3', 'wk_4', 'wk_5', 'wk_6']

1) Cross val functions

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
import numpy as np

def get_score(y, prediction):
    return np.sqrt(mean_squared_error(y, prediction.clip(0,20)))

def train_test_split(X, y, block):
    X_train = X.loc[dates < block]
    y_train = y.loc[dates < block]
    X_test = X.loc[dates == block]
    y_test = y.loc[dates == block]
    return X_train, X_test, y_train, y_test

def cross_val(model, X, y, block_range):
    scores = pd.DataFrame(columns=['train', 'test'], index=block_range)
    predictions = pd.Series(index=dates[dates.isin(block_range)], name='prediction')

    for block in block_range:
        m = clone(model)
        X_train, X_test, y_train, y_test = train_test_split(X, y, block)
        
        m.fit(X_train, y_train)
        
        pred_train = m.predict(X_train)
        score_train = get_score(y_train.values, pred_train)
        scores.loc[block, 'train'] = score_train
        
        pred_test = m.predict(X_test)
        score_test = get_score(y_test.values, pred_test)
        scores.loc[block, 'test'] = score_test
        predictions.loc[block] =  pred_test
        
        del m, X_train, y_train, X_test, y_test
        gc.collect()
        
        l.lap(str(block), '{:.2f}'.format(score_train), '{:.2f}'.format(score_test))
    return scores, predictions

2) Features & target

2) Lightgbm

In [26]:
from lightgbm import LGBMRegressor

lgb_params = {
    'feature_fraction': 0.75, 
    'nthread': -1, 
    'min_data_in_leaf': 2**7,
    'bagging_fraction': 0.75,
    'learning_rate': 0.03,
    'objective': 'mse',
    'num_leaves': 2**7,
    'bagging_freq': 1,
    'n_estimators': 100
    }

block_range = range(22,23)
lgb_model = LGBMRegressor(**lgb_params)
lgb_res, lgb_pred = cross_val(lgb_model, X, y, block_range)

total: 0:38:41 | section: 0:00:54 | 22 | 0.80 | 0.99


3) Ridge

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge_score, ridge_pred = cross_val(ridge, X, y block_range)

4) Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso_res, lasso_pred = cross_val(lasso, block_range)

5) Kneighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PCA

neigh = make_pipeline(PCA(n_components=32), KNeighborsRegressor(n_jobs=-1))
neigh_res, neigh_pred = cross_val(neigh, block_range)

In [17]:
def create_submission(prediction, save=False):
    test = pd.read_csv(catalog['test'])
    submission = pd.DataFrame(data={'ID': test.ID, 
                                    'item_cnt_month': prediction.clip(0,20)}
                             )
    if save:
        submission.to_csv('submission.csv', index=False)
    return submission

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 34)
lgb_model.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.75, bagging_freq=1, boosting_type='gbdt',
       colsample_bytree=1, feature_fraction=0.75, learning_rate=0.03,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_data_in_leaf=128, min_split_gain=0, n_estimators=100,
       nthread=-1, num_leaves=128, objective='mse', reg_alpha=0,
       reg_lambda=0, seed=0, silent=True, subsample=1,
       subsample_for_bin=50000, subsample_freq=1)

In [16]:
pred = lgb_model.predict(X_test)

In [24]:
create_submission(pred, True)

,ID,item_cnt_month
0,0,0.535855
1,1,0.202071
2,2,1.055191
3,3,0.512488
4,4,3.075977
5,5,0.656605
6,6,0.983450
7,7,0.134337
8,8,0.514052
9,9,0.826054
